
# Basic linear algebra in torchTT


This notebook is an introduction into the basic linar algebra operations that can be perfromed using the `torchtt` package.
The basic operations such as +,-,*,@,norm,dot product can be performed between `torchtt.TT` instances without computing the full format by computing the TT cores of the result.
One exception is the elementwise division between TT objects. For this, no explicit form of the resulting TT cores can be derived and therefore optimization techniques have to be employed (see the notebook `fast_tt_operations.ipynb`).

Imports

In [1]:
import torch as tn
try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT
    import torchtt as tntt

We will create a couple of tensors for the opperations that follow

In [2]:
N = [10,10,10,10]
o = tntt.ones(N)
x = tntt.randn(N,[1,4,4,4,1])
y = tntt.TT(tn.reshape(tn.arange(N[0]*N[1]*N[2]*N[3], dtype = tn.float64),N))
A = tntt.randn([(n,n) for n in N],[1,2,3,4,1])
B = tntt.randn([(n,n) for n in N],[1,2,3,4,1])

### Addition

The TT class has the "+" operator implemeted. It performs the addition between TT objects (must have compatible shape and type) and it returns a TT object. 
One can also add scalars to a TT object (float/int/torch.tensor with 1d).

The TT rank of the result is the sum of the ranks of the inputs. This is usually an overshoot and rounding can decrease the rank while maintaining the accuracy.

Here are a few examples:

In [3]:
z = x+y 
print(z)
# adding scalars is also possible
z = 1+x+1.0
z = z+tn.tensor(1.0)
# it works for the TT amtrices too
M = A+A+1 
print(M)

TT with sizes and ranks:
N = [10, 10, 10, 10]
R = [1, 6, 6, 6, 1]

Device: cpu, dtype: torch.float64
#entries 840 compression 0.084

TT-matrix with sizes and ranks:
M = [10, 10, 10, 10]
N = [10, 10, 10, 10]
R = [1, 5, 7, 9, 1]
Device: cpu, dtype: torch.float64
#entries 11200 compression 0.000112



### Subtraction

The "-" operator is also implemented in  the `torchtt.TT` class. It can be used similarily to "+" between 2 `torchtt.TT` objects and between a `torchtt.TT` and a scalar.
It can also be used as a negation.



In [4]:
v = x-y-1-0.5
C = A-B-3.14
w = -x+x
print(tn.linalg.norm(w.full()))

tensor(6.6390e-15, dtype=torch.float64)


### Multiplication (elementwise)

One can perform the elementwise multiplication $\mathsf{u}_{i_1...i_d} = \mathsf{x}_{i_1...i_d} \mathsf{y}_{i_1...i_d}$ between 2 tensors in the TT format without goin to full format.
The main issues of this is that the rank of the result is the product of the ranks of the input TT tensors.

In [5]:
u = x*y
print(u)

M2 = A*A

TT with sizes and ranks:
N = [10, 10, 10, 10]
R = [1, 8, 8, 8, 1]

Device: cpu, dtype: torch.float64
#entries 1440 compression 0.144



### Matrix vector product and matrix matrix product

* TT matrix and TT tensor: $(\mathsf{Ax})_{i_1...i_d} = \sum\limits_{j_1...j_d}\mathsf{A}_{i_1...i_d,j_1...j_d} \mathsf{x}_{j_1...j_d}$
* TT matrix and TT matrix: $(\mathsf{AB})_{i_1...i_d,k_1...k_d} = \sum\limits_{j_1...j_d}\mathsf{A}_{i_1...i_d,j_1...j_d} \mathsf{B}_{j_1...j_d,k_1...k_d}$

In [6]:
print(A@x)
print(A@B)
print(A@B@x)

TT with sizes and ranks:
N = [10, 10, 10, 10]
R = [1, 8, 12, 16, 1]

Device: cpu, dtype: torch.float64
#entries 3120 compression 0.312

TT-matrix with sizes and ranks:
M = [10, 10, 10, 10]
N = [10, 10, 10, 10]
R = [1, 4, 9, 16, 1]
Device: cpu, dtype: torch.float64
#entries 20000 compression 0.0002

TT with sizes and ranks:
N = [10, 10, 10, 10]
R = [1, 16, 36, 64, 1]

Device: cpu, dtype: torch.float64
#entries 29600 compression 2.96



Multiplication can be performed between a TT operator and a full tensor (in torch.tensor format) the result in this case is a full tn.tensor

In [7]:
print(A@tn.rand(A.N, dtype = tn.float64))

tensor([[[[-2.3288e+01,  5.0420e+00, -1.9414e+01,  ...,  1.2282e+02,
           -5.3476e+00, -8.2549e+01],
          [-6.7813e+01, -5.2084e-02, -7.8030e+00,  ...,  7.1054e+01,
            1.6544e+02, -8.1939e+01],
          [-1.1334e+02,  8.2145e+01,  1.1479e+02,  ..., -9.4729e+01,
            1.3620e+00,  8.5508e+01],
          ...,
          [-7.4297e+01,  3.7259e+01, -2.4047e+01,  ...,  8.3455e+00,
            1.2365e+02, -1.6801e+01],
          [ 8.1033e+01,  5.2163e+01, -6.0508e+00,  ..., -8.2501e+01,
            3.9886e+01,  6.2096e-01],
          [-7.1429e+01,  1.2495e+02,  1.1200e+02,  ..., -1.8928e+01,
            4.5814e+01,  2.9777e+01]],

         [[ 2.5583e+01, -3.3133e+01, -1.5700e+01,  ..., -5.8119e+01,
            1.9798e+01,  4.8671e+01],
          [-1.9729e+01, -7.5949e+00, -5.3085e+01,  ...,  3.0598e+01,
           -1.3227e+01, -3.6456e+00],
          [ 5.7865e+01, -5.8245e+01, -4.2705e+01,  ...,  4.1728e+01,
           -2.8628e+01, -5.4425e+01],
          ...,
     

### Kronecker product


For computing the Kronecker product one can either use the "**" operator or the method `torchtt.kron()`.

In [8]:
print(x**y)
print(A**A)

TT with sizes and ranks:
N = [10, 10, 10, 10, 10, 10, 10, 10]
R = [1, 4, 4, 4, 1, 2, 2, 2, 1]

Device: cpu, dtype: torch.float64
#entries 520 compression 5.2e-06

TT-matrix with sizes and ranks:
M = [10, 10, 10, 10, 10, 10, 10, 10]
N = [10, 10, 10, 10, 10, 10, 10, 10]
R = [1, 2, 3, 4, 1, 2, 3, 4, 1]
Device: cpu, dtype: torch.float64
#entries 4800 compression 4.8e-13



### Norm

Frobenius norm of a tensor $||\mathsf{x}||_F^2 = \sum\limits_{i_1,...,i_d} \mathsf{x}_{i_1...i_d}$ can be directly domputed from a TT decomposition.

In [9]:
print(y.norm())
print(A.norm())

tensor(577306.9677, dtype=torch.float64)
tensor(9717.7531, dtype=torch.float64)


### Dot product and summing along modes

One can sum alonf dimensions in `torchtt`. The function is `torchtt.TT.sum()` and can be used without arguments to sum along all dimensions, returning a scalar:

In [10]:
print('sum() result ', y.sum())
print('Must be equal to ', tn.sum(y.full()))

sum() result  tensor(49995000.0000, dtype=torch.float64)
Must be equal to  tensor(49995000.0000, dtype=torch.float64)


If a list of modes is additionally provided, the summing will be performed along the given modes and a `torchtt.TT` object is returned.

In [11]:
print(x.sum(1))
print(x.sum([0,1,3]))
print(A.sum([1,2]))

TT with sizes and ranks:
N = [10, 10, 10]
R = [1, 4, 4, 1]

Device: cpu, dtype: torch.float64
#entries 240 compression 0.24

TT with sizes and ranks:
N = [10]
R = [1, 1]

Device: cpu, dtype: torch.float64
#entries 10 compression 1.0

TT-matrix with sizes and ranks:
M = [10, 10]
N = [10, 10]
R = [1, 2, 1]
Device: cpu, dtype: torch.float64
#entries 400 compression 0.04



Dot product between 2 tensors is also possible using the function `tortchtt.dot()`.

In [12]:
print(tntt.dot(y,y))

tensor(3.3328e+11, dtype=torch.float64)


Dot product can be performed between 2 tensors of different mode lengths.
The modes alonnd the dot product is performed must be equal.
And they are given as a list of integers as an additional argument.
The modes given are relative to the first tensor.
The returned value is a `torchtt.TT` instance.

In [13]:
t1 = tntt.randn([4,5,6,7,8,9],[1,2,4,4,4,4,1])
t2 = tntt.randn([5,7,9],[1,3,3,1])
print(tntt.dot(t1,t2,[1,3,5]))

TT with sizes and ranks:
N = [4, 6, 8]
R = [1, 2, 12, 1]

Device: cpu, dtype: torch.float64
#entries 248 compression 1.2916666666666667



### Reshaping

Given a tensor in the TT format, one can reshape it similarily as in pytorch or numpy.
The method is `torchtt.reshape()` and it taks as argument a `torchtt.TT` object, the new shape, the relative accuracy epsilon and a maximum rank. The last 2 are optional.
The method also performs rounding up to the desired accuracy.


In [14]:
q = tntt.TT(tn.reshape(tn.arange(2*3*4*5*7*3, dtype = tn.float64),[2,3,4,5,7,3]))
# perform a series of reshapes
w = tntt.reshape(q,[12,10,21])
print(w)
w = tntt.reshape(w,[360,7])
print(w)
w = tntt.reshape(w,[2,3,4,5,7,3])
print('Error ',(w-q).norm()/q.norm())


TT with sizes and ranks:
N = [12, 10, 21]
R = [1, 3, 2, 1]

Device: cpu, dtype: torch.float64
#entries 138 compression 0.05476190476190476

TT with sizes and ranks:
N = [360, 7]
R = [1, 5, 1]

Device: cpu, dtype: torch.float64
#entries 1835 compression 0.7281746031746031

Error  tensor(2.0166e-15, dtype=torch.float64)


Reshape works also for TT matrices. However there are some restrictions such as the merging or spliting of the dimensions must happen within the same core for both row/column indices.

In [15]:
A = tntt.randn([(4,8),(6,4),(5,6),(8,8)],[1,2,3,2,1])
B = tntt.reshape(A,[(2,4),(6,4),(10,12),(8,8)])
print(B)
B = tntt.reshape(B,[(60,32),(16,48)])
print(B)
B = tntt.reshape(B,[(4,8),(6,4),(5,6),(8,8)])
print('Error ',(B-A).norm()/A.norm())

# this will not work: tntt.reshape(A,[(24,4),(5,16),(8,24)])

TT-matrix with sizes and ranks:
M = [2, 6, 10, 8]
N = [4, 4, 12, 8]
R = [1, 8, 12, 2, 1]
Device: cpu, dtype: torch.float64
#entries 5376 compression 0.0036458333333333334

TT-matrix with sizes and ranks:
M = [60, 16]
N = [32, 48]
R = [1, 23, 1]
Device: cpu, dtype: torch.float64
#entries 61824 compression 0.04192708333333333

Error  tensor(5.3723e-15, dtype=torch.float64)
